# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os

import joblib, pickle

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
#ws = Worksapce.get(name="")

experiment_name = 'your experiment name here'
experiment=Experiment(workspace=ws, name=experiment_name)

run=experiment.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_cluster = 'hyper-rf-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [ ]:
compute_trgets = ws.compute_targets
for comp_name,comp_type in compute_trgets.items():
    print(comp_name,comp_type.type,comp_type.provisioning_state)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--n_estimators' : choice(100,150,250,300),
    '--max_depth': choice(3,5,7),
    '--min_samples_split': choice(2,3,4,5)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    entry_script='model_training.py',
    source_directory='.',
    compute_target=comp_trget)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=21,
    max_concurrent_runs=4,
    policy=early_termination_policy,
    estimator=estimator)

In [ ]:
#TODO: Submit your experiment
hyperdrive_submission = exp.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_submission.get_best_run_by_primary_metric() #Best Run
best_run.get_details()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

In [ ]:
#TODO: Save the best model
final_model = best_run.register_model(model_name = 'hyper_rf_model',model_path = 'model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
comp_trget.delete()